In [ ]:
%%capture capt
!pip install scikit-learn-extra --upgrade
!pip install -U scikit-learn --upgrade

In [ ]:
%%capture capt
import numpy as np
from sklearn.cluster import *
from sklearn.metrics import silhouette_score
from sklearn_extra.cluster import KMedoids
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from time import time

# Useful functions

In [ ]:
def display_time(start, stop):
    total = stop - start
    hours = total // 3600
    minutes = (total - 3600*hours) // 60
    seconds = int((total - 3600*hours) % 60)
    
    if hours > 0:
        print(f"Process took {hours}h, {minutes}min, {seconds}s to run")
    elif minutes > 0:
        print(f"Process took {minutes}min, {seconds}s to run")
    else:
        print(f"Process took {seconds}s to run")

# Importation data

In [ ]:
base_patient = pd.read_csv("../data/profil_patient.csv")

In [ ]:
%%capture capt
p_soins = pd.read_csv("../data/parcours_soins.csv")

# Data Pre-processing

In [ ]:
base_patient.Mort = base_patient.Mort.astype(int)
base_patient.CHOC = base_patient.CHOC.astype(int)
base_patient.BEN_SEX_COD = base_patient.BEN_SEX_COD-1

In [ ]:
# on ne conserve que l'annee de la premiere hospitalisation

base_patient.date_h0 = pd.to_datetime(base_patient.date_h0).apply(lambda date: int(date.year))

# Vecteur sur lequel appliquer le clustering

In [ ]:
items_to_keep = ["date_h0", "y_nais", "BEN_SEX_COD", "Mort", "Nb_survie", "Nb_hospit", "CHOC"]

In [ ]:
X_patient = base_patient[items_to_keep]

In [ ]:
X_patient.head(3)

# K-Means/K-Medoids

In [ ]:
import warnings
warnings.simplefilter("ignore")

In [ ]:
def get_model(X, method, n_clusters, agglo_metric=None, agglo_link='ward', fit=True, show_fit=False):
 
    if method == 'kmeans':
        model = KMeans(n_clusters = n_clusters, n_init='auto')
    elif method == 'kmedoids':
        model = KMedoids(n_clusters=n_clusters, init='k-medoids++')
    elif method == 'kmedoids_precomputed':
        model = KMedoids(n_clusters=n_clusters, metric='precomputed', init='k-medoids++')
    elif method == 'bisecting_kmeans':
        model = BisectingKMeans(n_clusters=n_clusters, init='k-means++')
    elif method == 'agglomerative':
        model = AgglomerativeClustering(n_clusters=n_clusters, metric=agglo_metric,
                                    linkage=agglo_link)
    else:
        print("method was not in list.")
        print("Choose in ['kmeans', 'kmedoids', 'kmedoids_precomputed', 'bisecting_kmeans', 'agglomerative']")
        return

    if fit:
        print("The model is being fitted") if show_fit else None
        start = time()
        model.fit(X)
        stop = time()
        display_time(start, stop) if show_fit else None
    return model

In [68]:
def bic_score(X, labels):

    n_points = len(labels)
    n_clusters = len(set(labels))
    n_dimensions = np.shape(X)[1]

    n_parameters = (n_clusters - 1) + (n_dimensions * n_clusters) + 1

    loglikelihood = 0
    for label_name in set(labels):
        X_cluster = X[labels == label_name]
        n_points_cluster = len(X_cluster)
        centroid = np.mean(X_cluster, axis=0)
        variance = np.sum((X_cluster - centroid) ** 2) / (len(X_cluster) - 1)
        loglikelihood += \
          n_points_cluster * np.log(n_points_cluster) \
          - n_points_cluster * np.log(n_points) \
          - n_points_cluster * n_dimensions / 2 * np.log(2 * np.pi * variance) \
          - (n_points_cluster - 1) / 2

        bic = loglikelihood - (n_parameters / 2) * np.log(n_points)

    return bic

In [72]:
def calculate_scores(X, model):
    results = {}
    try:
        results["inertia"] = model.inertia_
    except:
        results["inertia"] = float('inf')
        print(f"This model {model} has no inertia")
    
    try:
        results["silhouette"] = silhouette_score(X_patient, model.labels_)
    except:
        results["silhouette"] = float('inf')
    
    try:
        results["bic"] = bic_score(X_patient.values, model.labels_)
    except:
        results["bic"] = -float('inf')
    return results

In [116]:
%%capture capt
!pip install kneed[plot]

from kneed import KneeLocator

def find_elbow(inertia):
    k_elbow = KneeLocator(
      x=range(2, len(inertia)+2), 
      y=inertia, 
      curve="convex", 
      direction="decreasing").elbow
    return k_elbow

In [117]:
def calculate_scores_all_clusters(X, method, max_n_cluster = int(np.sqrt(len(X_patient))), agglo_metric=None, agglo_link='ward'):
    inertia = []
    bic = []
    silhouette = []
   
    for n_clusters in tqdm(range(2, max_n_cluster)):
        model = get_model(X, method, n_clusters, agglo_metric, agglo_link, fit=True)
        scores = calculate_scores(X, model)
        inertia.append(scores["inertia"])
        bic.append(scores["bic"])
        silhouette.append(scores["silhouette"])
                
    return inertia, bic, silhouette

In [129]:
def plot_scores(method, plots, names):
    
    for i, name in enumerate(names):
        if name == "inertia" or name == "inertie":
            elbow = find_elbow(plots[i])
            print(f"La méthode du coude --> nombre de clusters = {elbow} pour le modele {method}\n")
    
    fig, ax = plt.subplots(1, len(plots), figsize=(6*len(plots), 6))
    plt.suptitle(f"Plots for method {method}")
    for i, plot in enumerate(plots):
        ax[i].plot(range(2, len(plot)+2), plot, c = 'r', label = str(names[i]), marker='o')
        ax[i].legend(loc='best')
        ax[i].grid('on')
        ax[i].set_xlabel('Nombre de clusters')
        ax[i].set_ylabel('Score = ' + str(names[i]))
        
    plt.show()

In [147]:
def do_all(X, method, max_n_cluster = int(np.sqrt(len(X_patient))), agglo_metric=None, agglo_link='ward'):
    inertia, bic, silhouette = calculate_scores_all_clusters(X, method, max_n_cluster, agglo_metric, agglo_link)
    
    plots, names = [], []
    if len(np.unique(inertia)) > 1:
        plots.append(inertia)
        names.append("inertia")
    if len(np.unique(bic)) > 1:
        plots.append(bic)
        names.append("bic")
    if len(np.unique(silhouette)) > 1:
        plots.append(silhouette)
        names.append("silhouette")
    
    plot_scores(method, plots, names)

In [ ]:
do_all(X_patient, 'agglomerative', max_n_cluster=15)